In [4]:
#@title Packages and functions
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import jv, jvp, yv, iv
from scipy.optimize import root_scalar
plt.rcParams.update({'font.size': 22})
# %matplotlib qt

def Cylinder_modes(cL, cT, R, order, modes):
  def Viktorov_matrix(omega, p , cl, ct, R):
      '''
      Equation I.28 can be written as a matrix.  Where the determinant of that matrix is zero, we have the roots
      of the equality.
      omega = frequency
      p = angular wavenumber
      cl = longitudinal wave velocity of material
      ct = shear wave velocity of material
      '''
      # re-scaling for numerical convenience
      R = R*1000
      cl = cl/1000
      ct = ct/1000
      omega = omega/1e6
      # Code starts
      kl = omega/cl
      kt = omega/ct
      x = kl*R
      y = kt*R
      # Matrix form
      a = (jv(p+2,x) + jv(p-2,x) - 2*((kt/kl)**2 -1)*jv(p,x))
      b = (jv(p+2,y) - jv(p-2,y))
      c = (jv(p+2,x) - jv(p-2,x))
      d = (jv(p+2,y) + jv(p-2,y))
      M = np.array([[a, b],[c, d]])
      return M

  def D_det(omega):
      '''Calculate the determinant of matrix '''
      return np.linalg.det(Viktorov_matrix(omega, kp, cL, cT, R))
  # Define constants and range of calculations
  fstop = 5.0*1e6 #5.0 # Ending frequency in MHz
  dp = 1.0 #0.2 # 0.1
  wstart = 0.0001*1e6 #0.0001*1e6
  dw = 0.001*1e6 # 0.001*1e6 #0.001; #0.01
  kpstart = 0 #0.001 #0.001 #0.04*radius; #0.01
  kpstop = order #20.0#20.0 #kstop*radius;

  omegas = np.arange(wstart,2*np.pi*fstop,dw)
  kps = np.arange(kpstart,kpstop,dp)

  number_of_roots = modes

  roots = []
  KP = []
  for kp in kps: #kp
      print(kp)
      print('mode', 'n', 'freq (Hz)', 'ka')
      rc = 0
      for omega in omegas:
          if kp <= 1.0: #1.0:
              NR = number_of_roots #- 1 #
              if rc < NR: #number_of_roots:
                  if (np.sign(D_det(omega)) > np.sign(D_det(omega+dw))) or (np.sign(D_det(omega)) < np.sign(D_det(omega+dw))):
                      #print(omega)\
                      root_val = root_scalar(D_det, bracket = [omega,omega+dw], method = 'bisect', xtol = 1e-7)# root_val.root
                      #if rc == 0:
                      #   roots.append(0)
                      #   KP.append(kp)
                      if root_val != None:
                          roots.append(root_val.root)
                          KP.append(kp)
                          print(''+str(int(kp))+'_S_'+str(rc)+'', kp, root_val.root/(2*np.pi), root_val.root*R/cT)
                          rc = rc + 1
              elif rc == NR:
                  break
          elif kp > 1.0: #1.5:
              NR = number_of_roots #
              if rc < NR: #number_of_roots:
                  if (np.sign(D_det(omega)) > np.sign(D_det(omega+dw))) or (np.sign(D_det(omega)) < np.sign(D_det(omega+dw))):
                      #print(omega)\
                      root_val = root_scalar(D_det, bracket = [omega,omega+dw], method = 'bisect', xtol = 1e-7)# root_val.root
                      if root_val != None:
                          roots.append(root_val.root)
                          KP.append(kp)
                          print(''+str(int(kp))+'_S_'+str(rc)+'', kp, root_val.root/(2*np.pi), root_val.root*R/cT)
                          rc = rc + 1
                          if rc == 1:
                              wstart = root_val.root
                              omegas = np.arange(wstart,2*np.pi*fstop,dw)
              elif rc == NR:
                  break
  return roots, KP

# Calculate the fundamental eigenmodes of a cylinder

You can compare the values to the ones of Lucien Saviot [here.](https://saviot.cnrs.fr/cyl/index.en.html)

In [5]:
#@title Calculate fundamental modes

import ipywidgets as widgets
from IPython.display import display, clear_output

# Create input widgets
cL_widget = widgets.FloatText(value=1000.0, description='cL (m/s):')
cT_widget = widgets.FloatText(value=36.0, description='cT (m/s):')
R_widget = widgets.FloatText(value=0.02627, description='R (m):')
order_widget = widgets.IntText(value=3, description='Order:')
modes_widget = widgets.IntText(value=10, description='Modes:')

# Create button widget
button = widgets.Button(description="Calculate Modes")

# Create output widget
out = widgets.Output()

# Define on_button_clicked function
def on_button_clicked(b):
  with out:
    clear_output()
    roots, KP = Cylinder_modes(cL_widget.value, cT_widget.value, R_widget.value, order_widget.value, modes_widget.value)
  return roots, KP

# Attach on_click event to button
button.on_click(on_button_clicked)

# Display widgets
display(order_widget, modes_widget ,cL_widget, cT_widget, R_widget, button, out)

roots, KP = on_button_clicked(button)

IntText(value=3, description='Order:')

IntText(value=10, description='Modes:')

FloatText(value=1000.0, description='cL (m/s):')

FloatText(value=36.0, description='cT (m/s):')

FloatText(value=0.02627, description='R (m):')

Button(description='Calculate Modes', style=ButtonStyle())

Output()

In [6]:
len(np.array(roots)/(2*np.pi))

30